In [4]:
import os
import requests
from urllib.parse import quote
from openai import OpenAI

# ✅ 1. API 키 읽기
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
client = OpenAI(api_key=api_key)

# ✅ 2. 고캠핑 인증키
GCAMP_API_KEY_RAW = "kuS/fQ3ICSDrvgpnSru44lJUxbyKbVSYRdGvIRMnoaJer7cd9/h5u42tN7XWrwH4FitT3uqYz+PQfoejRZdmkg=="
GCAMP_API_KEY = quote(GCAMP_API_KEY_RAW, safe='') 

# ✅ 3. 캠핑장 검색 함수
def search_campsites(keyword: str, limit: int = 5):
    try:
        encoded_keyword = quote(keyword)  # 한글 인코딩
        url = (
            f"https://apis.data.go.kr/B551011/GoCamping/searchList"
            f"?serviceKey={GCAMP_API_KEY}&MobileOS=ETC&MobileApp=CampingTest"
            f"&_type=json&keyword={encoded_keyword}&numOfRows=3"
        )
        print(f"🔍 요청 중: {url}")  # 요청 URL 확인
        
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()
        
        items = data.get("response", {}).get("body", {}).get("items", {}).get("item", [])
        return items
    except requests.exceptions.Timeout:
        print("⏱️ 요청 시간이 초과되었습니다. 네트워크를 확인해주세요.")
        return []
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return []

# ✅ 4. 사용자 입력
try:
    user_input = input("🏕️ 캠핑장 조건을 입력하세요 (예: '강원도 바다 근처'): ")
except:
    user_input = "강원도 바다"
    print(f"[자동입력 대체]: {user_input}")

# ✅ 5. 캠핑장 검색
results = search_campsites(user_input, limit=5)

if not results:
    print("⚠️ 조건에 맞는 캠핑장을 찾을 수 없습니다.")
else:
    summaries = []
    for site in results:
        summaries.append(f"- {site.get('facltNm')} ({site.get('addr1')}) / 테마: {site.get('lctCl', '없음')}")

    prompt = f"""
다음은 '{user_input}' 조건으로 검색된 캠핑장 목록입니다:

{chr(10).join(summaries)}

이 중 초보자나 가족 단위 여행자에게 적합한 2~3곳을 추천하고 간단한 이유를 설명해 주세요.
"""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 캠핑장 추천 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=600
    )

    print("\n🏕️ ChatGPT 캠핑장 추천 결과:")
    print("=" * 60)
    print(response.choices[0].message.content)

🔍 요청 중: https://apis.data.go.kr/B551011/GoCamping/searchList?serviceKey=kuS%2FfQ3ICSDrvgpnSru44lJUxbyKbVSYRdGvIRMnoaJer7cd9%2Fh5u42tN7XWrwH4FitT3uqYz%2BPQfoejRZdmkg%3D%3D&MobileOS=ETC&MobileApp=CampingTest&_type=json&keyword=%EA%B0%95%EC%9B%90%EB%8F%84%20%EB%B0%94%EB%8B%A4&numOfRows=3
❌ 오류 발생: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /B551011/GoCamping/searchList?serviceKey=kuS%2FfQ3ICSDrvgpnSru44lJUxbyKbVSYRdGvIRMnoaJer7cd9%2Fh5u42tN7XWrwH4FitT3uqYz%2BPQfoejRZdmkg%3D%3D&MobileOS=ETC&MobileApp=CampingTest&_type=json&keyword=%EA%B0%95%EC%9B%90%EB%8F%84%20%EB%B0%94%EB%8B%A4&numOfRows=3 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:1007)')))
⚠️ 조건에 맞는 캠핑장을 찾을 수 없습니다.
